In [1]:
%load_ext autoreload

%autoreload 3

In [2]:
%reload_ext autoreload

In [9]:
!pip install wget
!python -m wget https://chromedriver.storage.googleapis.com/108.0.5359.71/chromedriver_win32.zip

You should consider upgrading via the 'C:\Users\user\.virtualenvs\google-arts-0o2XoGZ3\Scripts\python.exe -m pip install --upgrade pip' command.



Saved under chromedriver_win32.zip


In [3]:
import src.scraping.scraping_pictures 
print(dir(src.scraping.scraping_pictures ))

['ScrapingSection', 'Union', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'pd', 'scripts', 'time', 'webdriver']


In [13]:
import pyprojroot
from src.scraping.scraping_pictures import ScrapingSection
import time

In [23]:
root_path =  pyprojroot.here()
path_driver = root_path / "notebooks"/'chromedriver.exe'

In [7]:

scraper = ScrapingSection(path_driver)
scraper.open("WHITE")
scraper.scroll_down(0)
#time.sleep(2)
df = scraper.scrape()
#print(src)

Open page section WHITE
Finish scrolling down
                                                image  \
0   url("https://lh3.googleusercontent.com/ci/AC_F...   
1   url("https://lh3.googleusercontent.com/ci/AC_F...   
2   url("https://lh3.googleusercontent.com/ci/AC_F...   
3   url("https://lh3.googleusercontent.com/ci/AC_F...   
4   url("https://lh3.googleusercontent.com/ci/AC_F...   
5   url("https://lh3.googleusercontent.com/ci/AC_F...   
6   url("https://lh3.googleusercontent.com/ci/AC_F...   
7   url("https://lh3.googleusercontent.com/ci/AC_F...   
8   url("https://lh3.googleusercontent.com/ci/AC_F...   
9   url("https://lh3.googleusercontent.com/ci/AC_F...   
10  url("https://lh3.googleusercontent.com/ci/AC_F...   
11  url("https://lh3.googleusercontent.com/ci/AC_F...   
12                                               none   
13                                               none   
14                                               none   
15                                        

In [8]:
df = scraper.scrape()

In [10]:
df.to_csv('')

,image,page
0,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/the-ma...
1,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/sympho...
2,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/the-cr...
3,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/the-cr...
4,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/portra...
5,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/summer...
6,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/ram-s-...
7,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/compos...
8,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/mother...
9,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/market...


In [17]:
import pyprojroot

pyprojroot.here()


PosixPath('/home/x/Desktop/portfolio/machine_learning/google-arts')